In [3]:
import AFQ.data as afqd
import cloudknot as ck
import importlib
import s3fs

/Users/richford/miniconda3/envs/scratch_pyAFQ_hbn/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/richford/miniconda3/envs/scratch_pyAFQ_hbn/lib/python3.7/site-packages/dipy/stats/__init__.py:7: UserWarning: The `dipy.stats` module is still under heavy development and functionality, as well as the API is likely to change in future versions of the software
  warnings.warn(w_string)
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credential

In [42]:
importlib.reload(afqd)

<module 'AFQ.data' from '/Users/richford/projects/neuro/afq/pyAFQ/AFQ/data.py'>

In [46]:
ixi = afqd.S3BIDSStudy(
    study_id='ixi', bucket='ixi.bids', s3_prefix='', subjects=5,
    anon=False, use_participants_tsv=False, random_seed=42
)

Retrieving subject S3 keys
[########################################] | 100% Completed |  1.9s


In [81]:
ixi.non_sub_s3_keys

{'raw': ['ixi.bids/dataset_description.json',
  'ixi.bids/dwi.bval',
  'ixi.bids/dwi.bvec'],
 'derivatives': []}

In [82]:
import os.path as op

for fn in ixi.non_sub_s3_keys['raw']:
    if 'bval' in fn or 'bvec' in fn:
        fs.get(fn, op.join('.', 'ixi-bids', op.basename(fn)))

In [50]:
ixi.subjects

[S3BIDSSubject(subject_id=sub-IXI108, study_id=ixi, site=None,
 S3BIDSSubject(subject_id=sub-IXI039, study_id=ixi, site=None,
 S3BIDSSubject(subject_id=sub-IXI139, study_id=ixi, site=None,
 S3BIDSSubject(subject_id=sub-IXI631, study_id=ixi, site=None,
 S3BIDSSubject(subject_id=sub-IXI538, study_id=ixi, site=None]

In [61]:
s0 = ixi.subjects[0]
s1 = ixi.subjects[1]

In [57]:
ixi.download('./ixi-bids')

In [58]:
s0.files

{'raw': {'sub-IXI108/anat/sub-IXI108_T1w.nii.gz': '/Users/richford/projects/neuro/afq/pyAFQ/ixi-bids/sub-IXI108/anat/sub-IXI108_T1w.nii.gz',
  'sub-IXI108/dwi/sub-IXI108_dwi.nii.gz': '/Users/richford/projects/neuro/afq/pyAFQ/ixi-bids/sub-IXI108/dwi/sub-IXI108_dwi.nii.gz'},
 'derivatives': {}}

In [63]:
import os.path as op

op.commonpath(list(s0.files['raw'].values()) + list(s1.files['raw'].values()))

'/Users/richford/projects/neuro/afq/pyAFQ/ixi-bids'

In [83]:
def preprocess_ixi_subject(subject_id):
    import AFQ.data as afqd
    import os
    import subprocess
    from s3fs import S3FileSystem
    
    ixi = afqd.afqd.S3BIDSStudy(
        study_id='ixi', bucket='ixi.bids', s3_prefix='', subjects=subject_id,
        anon=False, use_participants_tsv=False
    )
        
    ixi.download("./ixi")
    
    fs = s3fs.S3FileSystem()
        
    for fn in ixi.non_sub_s3_keys['raw']:
        if 'bval' in fn or 'bvec' in fn:
            fs.get(fn, os.path.join('.', 'ixi-bids', op.basename(fn)))
    
    subprocess.run(
        ["qsiprep",
         "./ixi",
         "./ixi-preproc",
         subject_id],
        shell=True,
        check=True,
    )

    def upload_dir(path):
        output_dir = 'ixi.bids/derivatives/qsiprep_ck'
        if fs.exists(output_dir):
            walk_dir = os.walk(path)
            _, dirs, files = next(walk_dir)
            for fn in dirs + files:
                fs.put(os.path.join(path, fn), output_dir, recursive=True)
        else:
            fs.put(path, output_dir, recursive=True)
            
    upload_dir("./ixi-preproc")

In [84]:
di = ck.DockerImage(name="preprocess-ixi-subject")

In [86]:
di.build(tags="ixi_preproc_20200708")

In [ ]:
knot = ck.Knot(
    name="ixi_preproc_20200708_v2",
#     func=preprocess_ixi_subject,
#     base_image="pennbbl/qsiprep",
    docker_image=di,
#     image_github_installs="https://github.com/richford/pyAFQ.git@s3-bids-fetch",
    pars_policies=('AmazonS3FullAccess',),
    resource_type="SPOT",
    bid_percentage=100,
    memory=64000,
    job_def_vcpus=32,
)

In [ ]:
sub_ids = [s.subject_id for s in hbn.subjects]
sub_ids

In [ ]:
result_futures = knot.map(sub_ids)

In [ ]:
knot.view_jobs()

In [ ]:
print([j.status for j in knot.jobs])

In [ ]:
knot.clobber(clobber_pars=True)